In [1]:
import pickle
import pandas as pd
import deepchem as dc
import torch
from torch_geometric.data import Data
from sklearn.model_selection import train_test_split

C:\Users\noton\miniconda3\envs\Deep2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'dgl'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
torch.manual_seed(0)
df = pd.read_csv('data_zinc_50572/zinc_50572.csv')
df

,Substance ID,SMILES,BertzCT,Kappa2
0,ZINC000000000007,C=CCc1ccc(OCC(=O)N(CC)CC)c(OC)c1,453.035249,8.250757
1,ZINC000000000010,C[C@@]1(c2ccccc2)OC(C(=O)O)=CC1=O,475.051945,3.483475
2,ZINC000000000011,COc1cc(Cc2cnc(N)nc2N)cc(OC)c1N(C)C,647.972650,6.379815
3,ZINC000000000012,O=C(C[S@@](=O)C(c1ccccc1)c1ccccc1)NO,547.082326,6.977663
4,ZINC000000000014,CC[C@H]1[C@H](O)N2[C@H]3C[C@@]45c6ccccc6N(C)[C...,723.345329,4.169324
...,...,...,...,...
50567,ZINC000000073943,CC(C)Cc1nnc(NS(=O)(=O)c2ccc(Cl)cc2)s1,680.982927,6.069763
50568,ZINC000000073944,C[C@H](CN1CCCCC1)OC(=O)c1ccc2c(c1)OCCO2,525.390265,7.006574
50569,ZINC000000073946,COc1cc(C(=O)N[C@H](C(=O)O)C(C)C)cc(OC)c1OC,529.958808,7.534745
50570,ZINC000000073948,COc1cc(C(=O)N[C@@H](C(=O)O)C(C)C)cc(OC)c1OC,529.958808,7.534745


In [3]:
torch.manual_seed(0)

smiles = df["SMILES"]
featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
X = featurizer.featurize(smiles)

In [4]:
torch.manual_seed(0)

y_bertzct = df["BertzCT"]
data_train, data_temp, target_train_bertzct, target_temp_bertzct = train_test_split(X, y_bertzct, train_size=0.7, random_state=0)
data_val, data_test, target_val_bertzct, target_test_bertzct = train_test_split(data_temp, target_temp_bertzct, train_size=0.5, random_state=0)

target_train_bertzct = torch.tensor(target_train_bertzct.values, dtype=torch.float32)
target_val_bertzct = torch.tensor(target_val_bertzct.values, dtype=torch.float32)
target_test_bertzct = torch.tensor(target_test_bertzct.values, dtype=torch.float32)

data_train_list = []
for graph_data in data_train:
    node_features = torch.tensor(graph_data.node_features, dtype=torch.float32)
    edge_index = torch.tensor(graph_data.edge_index, dtype=torch.long)
    edge_features = torch.tensor(graph_data.edge_features, dtype=torch.float32)
    data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)
    data_train_list.append(data)
    
data_val_list = []
for graph_data in data_val:
    node_features = torch.tensor(graph_data.node_features, dtype=torch.float32)
    edge_index = torch.tensor(graph_data.edge_index, dtype=torch.long)
    edge_features = torch.tensor(graph_data.edge_features, dtype=torch.float32)
    data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)
    data_val_list.append(data)

data_test_list = []
for graph_data in data_test:
    node_features = torch.tensor(graph_data.node_features, dtype=torch.float32)
    edge_index = torch.tensor(graph_data.edge_index, dtype=torch.long)
    edge_features = torch.tensor(graph_data.edge_features, dtype=torch.float32)
    data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)
    data_test_list.append(data)

In [5]:
torch.manual_seed(0)

with open('data_zinc_50572/data_train_list.pkl', 'wb') as f:
    pickle.dump(data_train_list, f)
with open('data_zinc_50572/data_val_list.pkl', 'wb') as f:
    pickle.dump(data_val_list, f)
with open('data_zinc_50572/data_test_list.pkl', 'wb') as f:
    pickle.dump(data_test_list, f)
with open('data_zinc_50572/target_train_bertzct.pkl', 'wb') as f:
    pickle.dump(target_train_bertzct, f)
with open('data_zinc_50572/target_val_bertzct.pkl', 'wb') as f:
    pickle.dump(target_val_bertzct, f)
with open('data_zinc_50572/target_test_bertzct.pkl', 'wb') as f:
    pickle.dump(target_test_bertzct, f)